In [1]:
import gc
import pandas as pd
from sklearn.metrics import accuracy_score, \
    precision_score, \
    recall_score, \
    ConfusionMatrixDisplay, \
    f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt

In [2]:
PATH = "../data"
DATASET = 'news_29_80k'

data = pd.read_csv(f"{PATH}/dataset/{DATASET}.csv").dropna()

In [4]:
import nltk 
nltk.data.path.append("../requirements/nltk_data/")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [14]:
from sklearn.model_selection import train_test_split

X = data.drop(["id", "channel", "genre"], axis=1)
y = data.drop(["id", "channel", "text"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=.2)

stemmer = SnowballStemmer(language="russian")
stop_words = stopwords.words("russian")

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Eugeny Kats/nltk_data'
    - 'c:\\Working directory\\ml-projects\\aiNews\\.venv\\nltk_data'
    - 'c:\\Working directory\\ml-projects\\aiNews\\.venv\\share\\nltk_data'
    - 'c:\\Working directory\\ml-projects\\aiNews\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\Eugeny Kats\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:

def text_to_tokens(text: str) -> list:
    tokens = word_tokenize(text, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    tokens = [i for i in tokens if i not in stop_words]
    tokens = [stemmer.stem(i) for i in tokens]
    return tokens


vectorizer = TfidfVectorizer(tokenizer=text_to_tokens)
X_train = vectorizer.fit_transform(X_train['text'])
X_test = vectorizer.transform(X_test['text'])
gc.collect()

tone_classifier = LogisticRegression(
    penalty="elasticnet", random_state=42, C=.9, solver="saga", l1_ratio=.4)
tone_model = tone_classifier.fit(X_train, y_train)
y_pred = tone_model.predict(X_test)


In [ ]:
import scikitplot as skplt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_pred))
print(precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
print(recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
print(accuracy_score(y_true=y_test, y_pred=y_pred))
ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=y_pred)
plt.show()